In [1]:
import pkg_resources.py2_warn
import pandas as pd
import numpy as np
import os

from pathlib import Path
from datetime import datetime

In [2]:
'''
This script takes in a Freight Matrix file in the same directory as the executable,
    and creates another file f'Carrier Choice - {date_string}.xlsx' which yields the
    top 5 choices by cost (i.e. 1st choice = lowest cost)
'''

"\nThis script takes in a Freight Matrix file in the same directory as the executable,\n    and creates another file f'Carrier Choice - {date_string}.xlsx' which yields the\n    top 5 choices by cost (i.e. 1st choice = lowest cost)\n"

In [3]:
# Get BAF Month from user
print('Please type in the 3-letter BAF month. (e.g. jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec)')
baf_month = input()

allowed_months = ['jan', 'feb', 'mar', 'apr', 'may',
                  'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
if baf_month not in allowed_months:
    error = 'The month you have indicated is not permissible, please check your spelling.'
    print(error)
    input()
    raise Exception(error)

Please type in the 3-letter BAF month. (e.g. jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec)
aug


In [4]:
# Read the Freight Matrix
print('Reading Freight Matrix file...this action can take up to 2 minutes.')
files = list(Path(os.getcwd()).glob('Freight Matrix *.xlsx'))
if len(files) == 1:
    df = pd.read_excel(files[0].name, header=1)
else:
    error = 'There are multiple Freight Matrix files that fit the pattern, or no suitable Freight Matrix files! Please include only one in the same directory as this script with the naming pattern of "Freight Matrix - *.xlsx".'
    print(error)
    input()
    raise FileExistsError(error)

Reading Freight Matrix file...this action can take up to 2 minutes.


In [5]:
print('Building carrier choice file...')

Building carrier choice file...


In [6]:
# Create additional identifier key columns
df['plant_pod_ct'] = df['Plant'] + df['Port of Discharge'] + df['Container Type']
df['plant_pod_carrier_ct'] = df['Plant'] + df['Port of Discharge'] + df['Carrier'] + df['Container Type']

# Filter by provided BAF Month, and drop
new_df = (df.loc[df['BAF Month'].str.lower() == baf_month]
            .sort_values('Total Logistics Cost USD')
            .drop_duplicates(['plant_pod_carrier_ct']))

In [8]:
# Group by port pairs and sort by total logs cost
grouped_df = (new_df
              .groupby('plant_pod_ct')
              .apply(pd.DataFrame.sort_values, 'Total Logistics Cost USD'))
grouped_df

Plant    Carrier Place of receipt  \
plant_pod_ct                                                        
Bell BayAUADLDMSTCLCL 40674  Bell Bay       TOLL         Bell Bay   
Bell BayAUBNEDMSTCFCL 40665  Bell Bay       TOLL         Bell Bay   
Bell BayAUBNEDMSTCLCL 40677  Bell Bay       TOLL         Bell Bay   
Bell BayAUGETDMSTCLCL 40671  Bell Bay       TOLL         Bell Bay   
Bell BayAUGEXDMSTCLCL 40692  Bell Bay       TOLL         Bell Bay   
...                               ...        ...              ...   
TomagoVNSGN20'GP      20840    Tomago      OOCLA           Sydney   
                      22339    Tomago  ONEXPRESS           Sydney   
                      17207    Tomago        MSC           Sydney   
                      40152    Tomago   EVERSHIA           Sydney   
                      30412    Tomago    HAMBURG           Sydney   

                            Port of Loading Port of Loading Country  \
plant_pod_ct                                                          
Bell BayAUADLDMSTCLCL 40674           AUBEL               AUSTRALIA   
Bell BayAUBNEDMSTCFCL 40665           AUBEL               AUSTRALIA   
Bell BayAUBNEDMSTCLCL 40677           AUBEL               AUSTRALIA   
Bell BayAUGETDMSTCLCL 40671           AUBEL               AUSTRALIA   
Bell BayAUGEXDMSTCLCL 40692           AUBEL               AUSTRALIA   
...                                     ...                     ...   
TomagoVNSGN20'GP      20840           AUSYD               AUSTRALIA   
                      22339           AUSYD               AUSTRALIA   
                      17207           AUSYD               AUSTRALIA   
                      40152           AUSYD               AUSTRALIA   
                      30412           AUSYD               AUSTRALIA   

                            Port of Loading Name Port of Discharge  \
plant_pod_ct                                                         
Bell BayAUADLDMSTCLCL 40674             Bell Bay             AUADL   
Bell BayAUBNEDMSTCFCL 40665             Bell Bay             AUBNE   
Bell BayAUBNEDMSTCLCL 40677             Bell Bay             AUBNE   
Bell BayAUGETDMSTCLCL 40671             Bell Bay             AUGET   
Bell BayAUGEXDMSTCLCL 40692             Bell Bay             AUGEX   
...                                          ...               ...   
TomagoVNSGN20'GP      20840               Sydney             VNSGN   
                      22339               Sydney             VNSGN   
                      17207               Sydney             VNSGN   
                      40152               Sydney             VNSGN   
                      30412               Sydney             VNSGN   

                            Port of Discharge Country Port of Discharge Name  \
plant_pod_ct                                                                   
Bell BayAUADLDMSTCLCL 40674                 AUSTRALIA               Adelaide   
Bell BayAUBNEDMSTCFCL 40665                 AUSTRALIA               Brisbane   
Bell BayAUBNEDMSTCLCL 40677                 AUSTRALIA               Brisbane   
Bell BayAUGETDMSTCLCL 40671                 AUSTRALIA              Geraldton   
Bell BayAUGEXDMSTCLCL 40692                 AUSTRALIA                Geelong   
...                                               ...                    ...   
TomagoVNSGN20'GP      20840                  VIET NAM       Ho Chi Minh City   
                      22339                  VIET NAM       Ho Chi Minh City   
                      17207                  VIET NAM       Ho Chi Minh City   
                      40152                  VIET NAM       Ho Chi Minh City   
                      30412                  VIET NAM       Ho Chi Minh City   

                             Mode  ... Sailing Frequency Tranship via  \
plant_pod_ct                       ...                                  
Bell BayAUADLDMSTCLCL 40674    61  ...               NaN          NaN   
Bell BayAUBNEDMSTCFCL 40665    61  ... 

In [9]:
# Drop the first multiIndex level to prevent index and column having same name
grouped_df.index = grouped_df.index.droplevel()
grouped_df[['plant_pod_ct', 'Total Logistics Cost USD', 'Carrier', 'Container Type']]

,plant_pod_ct,Total Logistics Cost USD,Carrier,Container Type
40674,Bell BayAUADLDMSTCLCL,1720.422633,TOLL,DMSTCLCL
40665,Bell BayAUBNEDMSTCFCL,2711.664118,TOLL,DMSTCFCL
40677,Bell BayAUBNEDMSTCLCL,5161.267899,TOLL,DMSTCLCL
40671,Bell BayAUGETDMSTCLCL,7906.992910,TOLL,DMSTCLCL
40692,Bell BayAUGEXDMSTCLCL,1772.556652,TOLL,DMSTCLCL
...,...,...,...,...
20840,TomagoVNSGN20'GP,744.118942,OOCLA,20'GP
22339,TomagoVNSGN20'GP,861.278942,ONEXPRESS,20'GP
17207,TomagoVNSGN20'GP,909.758942,MSC,20'GP
40152,TomagoVNSGN20'GP,1077.406445,EVERSHIA,20'GP


In [10]:
# Convert all values the Carrier column, within a group, to a series of lists
grouped_df = grouped_df.groupby('plant_pod_ct').agg({'Carrier': list})
grouped_df

,Carrier
plant_pod_ct,
Bell BayAUADLDMSTCLCL,[TOLL]
Bell BayAUBNEDMSTCFCL,[TOLL]
Bell BayAUBNEDMSTCLCL,[TOLL]
Bell BayAUGETDMSTCLCL,[TOLL]
Bell BayAUGEXDMSTCLCL,[TOLL]
...,...
TomagoUSSAV20'GP,"[CMACGMBRIS, MSC, HAMBURG, HAPAGLLOYD, ONEXPRE..."
TomagoVNCLI20'GP,"[HAPAGLLOYD, OOCLA, ONEXPRESS, EVERSHIA, HAMBURG]"
TomagoVNHPH20'GP,"[HAPAGLLOYD, OOCLA, ONEXPRESS, ANL, EVERSHIA, ..."


In [11]:
# Take only the first five choices
choices = ['1st choice', '2nd choice',
           '3rd choice', '4th choice', '5th choice']
grouped_df[choices] = pd.DataFrame(grouped_df['Carrier'].to_list(),
                                   index=grouped_df.index)[[0, 1, 2, 3, 4]]
grouped_df.drop('Carrier', inplace=True, axis=1)
grouped_df.reset_index(inplace=True)

In [12]:
# Assembling the final output
final_columns = ['Plant', 'Port of Loading',
                 'Port of Discharge', 'Port of Discharge Name', 'Container Type', 'BAF Month']

final_df = grouped_df.merge(new_df.drop_duplicates('plant_pod_ct')[
                            final_columns + ['plant_pod_ct']], on='plant_pod_ct')


final_df = final_df[final_columns + choices]

In [13]:
final_df

,Plant,Port of Loading,Port of Discharge,Port of Discharge Name,Container Type,BAF Month,1st choice,2nd choice,3rd choice,4th choice,5th choice
0,Bell Bay,AUBEL,AUADL,Adelaide,DMSTCLCL,Aug,TOLL,None,None,None,None
1,Bell Bay,AUBEL,AUBNE,Brisbane,DMSTCFCL,Aug,TOLL,None,None,None,None
2,Bell Bay,AUBEL,AUBNE,Brisbane,DMSTCLCL,Aug,TOLL,None,None,None,None
3,Bell Bay,AUBEL,AUGET,Geraldton,DMSTCLCL,Aug,TOLL,None,None,None,None
4,Bell Bay,AUBEL,AUGEX,Geelong,DMSTCLCL,Aug,TOLL,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
391,Tomago,AUSYD,USSAV,Savannah,20'GP,Aug,CMACGMBRIS,MSC,HAMBURG,HAPAGLLOYD,ONEXPRESS
392,Tomago,AUSYD,VNCLI,Cat Lai,20'GP,Aug,HAPAGLLOYD,OOCLA,ONEXPRESS,EVERSHIA,HAMBURG
393,Tomago,AUSYD,VNHPH,Haiphong,20'GP,Aug,HAPAGLLOYD,OOCLA,ONEXPRESS,ANL,EVERSHIA
394,Tomago,AUSYD,VNIPH,Phuoc Long ICD,20'GP,Aug,HAPAGLLOYD,ANL,OOCLA,ONEXPRESS,MSC


In [14]:
final_df.to_excel('Carrier Choice - additional.xlsx', index=False)

## Testing

In [ ]:
final_df.loc[(final_df['Port of Loading'] == 'AUBNE') & (final_df['Port of Discharge'] == 'CAVAN')]

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
grouped_df.loc[grouped_df.port_pair == 'AUBNECAVAN'][['Carrier','Total Logistics Cost USD']]